# File Search Agent

### Connect to the Foundry

In [ ]:
# Import packages
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # Load environment variables from .env file

# Use the connection string to connect to your Foundry project
project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string, credential=DefaultAzureCredential()
    )
except Exception as e:
    print(f"Failed to connect to project: {e}")
    raise


### Upload the files

In [ ]:
uploaded_file_ids = []

try:
    for local_file in os.listdir("data"):

        # Define the local file path
        local_file_path = os.path.join("data", local_file)

        print(f"Processing File '{local_file_path}'...")

        # Use the upload and poll SDK helper to upload the local file, add them to the vector store,
        # and poll the status of the file batch for completion.
        file = project.agents.upload_file_and_poll(file_path=local_file_path, purpose="assistants")
        uploaded_file_ids.append(file.id)

        print(f"Uploaded file, file ID: {file.id}")

            
    print(f"File IDs: {uploaded_file_ids}")

except FileNotFoundError:
    print(f"Error: The file '{curr_file}' was not found.")
    raise

except Exception as e:
    print(f"An error occurred while processing the Excel file: {e}")
    raise

### Create a vector store 

In [ ]:
# Create a vector store called "Financial Statements"
vector_store = project.agents.create_vector_store_and_poll(file_ids=uploaded_file_ids, name="financial_sample")
print(f"Vector store '{vector_store.name} ({vector_store.id})' created successfully.")

### Create the Agent with FileSearch Tool

In [ ]:
from azure.ai.projects.models import FileSearchTool

# Create file search tool with resources followed by creating agent
file_search = FileSearchTool(vector_store_ids=[vector_store.id])
print(f"File search tool created successfully for Vector Store {vector_store.name} ({vector_store.id}).")


## need to login using the command azd auth login
agent = project.agents.create_agent(
        model=os.getenv("CHAT_MODEL"),
        name="FileSearchAgent",
        instructions="You are helpful AI assistant.",
     
        tools=file_search.definitions,
        tool_resources=file_search.resources
    )
print(f"Created agent, agent ID: {agent.id}")

# Create a thread
thread = project.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

### Util method message handeling

In [ ]:
def sendMessage(msg):
    # Get messages from the thread
    message = project.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=msg
        )
    print(f"Created message, ID: {message.id}")

    run = project.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            print(f"Run failed: {run.last_error}")
            raise Exception(f"Run failed: {run.last_error}")

    messages = project.agents.list_messages(thread_id=thread.id, order="asc", after=message.id)
    for msg in messages.data:
        role = msg.role
        content = msg.content[0].text.value
        print(f"{role.capitalize()}: {content}")


In [ ]:
sendMessage("Who is Hogan Kathleen?")


In [ ]:
sendMessage("Each uploaded file is a form. Extract the name and the total number of stocks owned by the person. Output the results in a structured table format. The total number of stocks should be a numeric value.")